In [ ]:
from __future__ import annotations

from rerun_notebook import Viewer

Viewer(
    url="https://app.rerun.io/version/nightly/examples/raw_mesh.rrd",
    panel_states={
        "blueprint": "hidden",
        "selection": "hidden",
        "time": "collapsed",
    },
)